# IMPORTS

Model imports

In [5]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

KFold cross validation imports

In [13]:
import splitfolders
from sklearn.model_selection import KFold, StratifiedKFold

# Device

In [7]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Image transformation

In [15]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

# Data loader

In [14]:
splitfolders.ratio("../Data", output="../DataTrainTest", seed=1337, ratio=(.8, 0, .2), group_prefix=None, move=False)

Copying files: 2465 files [00:03, 623.86 files/s]


Separation no KFold cross

# KFold cross validation

In [9]:
data_dir = '../Data'

In [ ]:
folds = StratifiedKFold(n_splits=10)

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]

In [17]:
train_path='../DataTrainTest/train/'
test_path='../DataTrainTest/test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
for i, (images,labels) in enumerate(train_loader):
    print(images)

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

# Determine classes

In [18]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [19]:
print(classes)

['ClearSky', 'altocumulus', 'altostratus', 'cirrocumulus', 'cirrostratus', 'cirrus', 'cumulonimbus', 'cumulus', 'nimbostratus', 'stratocumulus', 'stratus']


# CNN Model

In [20]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=11):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [21]:
model=ConvNet(num_classes=11).to(device)

# Optimizer

In [22]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [23]:
num_epochs=10

In [58]:
train_count=len(glob.glob(train_path+'/**/*.jpeg')) + len(glob.glob(train_path+'/**/*.jpg')) + len(glob.glob(train_path+'/**/*.bmp')) + len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.jpeg')) + len(glob.glob(test_path+'/**/*.jpg')) + len(glob.glob(test_path+'/**/*.bmp')) + len(glob.glob(test_path+'/**/*.png'))

In [59]:
print(train_count,test_count)

1968 496


# Training

In [60]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(1.4420) Train Accuracy: 0.8460365853658537 Test Accuracy: 0.38911290322580644
Epoch: 1 Train Loss: tensor(1.9630) Train Accuracy: 0.7977642276422764 Test Accuracy: 0.35685483870967744
Epoch: 2 Train Loss: tensor(2.1323) Train Accuracy: 0.8140243902439024 Test Accuracy: 0.3971774193548387
Epoch: 3 Train Loss: tensor(1.5651) Train Accuracy: 0.8526422764227642 Test Accuracy: 0.35685483870967744
Epoch: 4 Train Loss: tensor(1.2642) Train Accuracy: 0.8760162601626016 Test Accuracy: 0.3911290322580645
Epoch: 5 Train Loss: tensor(1.3737) Train Accuracy: 0.8572154471544715 Test Accuracy: 0.36693548387096775
Epoch: 6 Train Loss: tensor(1.0827) Train Accuracy: 0.8765243902439024 Test Accuracy: 0.3709677419354839
Epoch: 7 Train Loss: tensor(1.2116) Train Accuracy: 0.858739837398374 Test Accuracy: 0.40524193548387094
Epoch: 8 Train Loss: tensor(1.2351) Train Accuracy: 0.8800813008130082 Test Accuracy: 0.3911290322580645
Epoch: 9 Train Loss: tensor(0.8949) Train Accuracy: